### Tips from prof

- Narrow scope of work (e.g. court level)

- Could try both binary/multi-class model outcomes and compare the performance 

- Change user from layperson to legal professional (and mention that this project is a stepping stone towards having layperson use the model)

- Link features to predicted outcome (if time permits can try using XGBoost with LIME for model interpretability)

- Can also try to see accuracy of models with different areas of law, lowest accuracy may be hardest area of law to predict


### Data setup

In [1]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import nltk
import pyLDAvis
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Load CSV files into DataFrames
areas_of_law_df = pd.read_csv("data/prediction_data/areas_of_law.csv")
coram_df = pd.read_csv("data/prediction_data/coram.csv")
fact_themes_df = pd.read_csv("data/prediction_data/fact_themes.csv")
sg_legal_cases_df = pd.read_csv("data/prediction_data/sg_legal_cases_dataset.csv")
target_rulings_df = pd.read_csv("data/prediction_data/target_rulings.csv")
# Load the JSON file into a dictionary
with open('data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
processed_df = pd.DataFrame(issues_data)

# Load the JSON file into a dictionary
with open('data/rawish_data/facts.json') as f:
    facts_data = [json.loads(line) for line in f]
raw_facts_df = pd.DataFrame(facts_data)

# Merge DataFrames
merged_df = pd.merge(areas_of_law_df, coram_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, fact_themes_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, sg_legal_cases_df, left_on='casename', right_on='filename', how='outer')
merged_df = pd.merge(merged_df, processed_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, raw_facts_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, target_rulings_df, on='casename', how='outer')

merged_df.drop(columns=['Unnamed: 0'], inplace=True)
merged_df.drop(columns=['filename'], inplace=True)

# Display the resulting DataFrame
print(merged_df.head())

           casename                                        area_of_law  \
0   2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1  2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2  2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3  2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4  2000_SGCA_13.pdf                     {'criminal law': ['offences']}   

                                           Coram  themes court_level  \
0  ['Chao Hick Tin , L P Thean , Yong Pung How']     5.0        SGCA   
1   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
2   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
3   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
4   ['Chao Hick Tin , Lai Kew Chai , L P Thean']    12.0        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the cla

In [3]:
merged_df = merged_df.dropna()
merged_df.isna().sum()

casename       0
area_of_law    0
Coram          0
themes         0
court_level    0
issues         0
facts          0
target         0
dtype: int64

### Data Preprocessing

In [4]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

#nas are probably those reassigned cases, coram has 7, i just drop them for now
na_target_rows = merged_df[merged_df['target'].isna()]
print(na_target_rows)

merged_df.dropna(axis=0, inplace=True)
print(merged_df.isna().sum())

#remove empty lists
merged_df = merged_df.query("area_of_law != '[]'")

#target is unbalanced
target_counts = merged_df['target'].value_counts()
print(target_counts)

merged_df = merged_df.reset_index(drop=True) # prevent nan values from appearing after one-hot

casename       0
area_of_law    0
Coram          0
themes         0
court_level    0
issues         0
facts          0
target         0
dtype: int64
Empty DataFrame
Columns: [casename, area_of_law, Coram, themes, court_level, issues, facts, target]
Index: []
casename       0
area_of_law    0
Coram          0
themes         0
court_level    0
issues         0
facts          0
target         0
dtype: int64
target
Favourable      3942
Unfavourable    2056
No outcome       795
Name: count, dtype: int64


In [5]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(ast.literal_eval)
merged_df['Coram'] = merged_df['Coram'].apply(ast.literal_eval)
merged_df.head(3)

,casename,area_of_law,Coram,themes,court_level,issues,facts,target
0,2000_SGCA_1.pdf,"{'civil procedure': ['pleadings'], 'res judica...","[Chao Hick Tin , L P Thean , Yong Pung How]",5.0,SGCA,The claim was dismissed with costs by the\nHig...,The facts\nThe appellant is the widow of one T...,Favourable
1,2000_SGCA_10.pdf,"{'contract': ['formation'], 'equity': ['defenc...","[Chao Hick Tin , Tan Lee Meng , L P Thean]",5.0,SGCA,the claim and\nagainst that decision this appe...,facts and surrounding circumstances including ...,Favourable
2,2000_SGCA_11.pdf,"{'contract': ['discharge'], 'damages': ['asses...","[Chao Hick Tin , Tan Lee Meng , L P Thean]",5.0,SGCA,The appeal \nThe questions which arise in this...,"Background \nThe first appellants, a French co...",No outcome


### Flatten areas_of_law

In [6]:
all_areas = []

for index, row in merged_df.iterrows():

    areas = row['area_of_law']
    flat_areas = []
    for main_area, sub_areas in areas.items():
        flat_areas.append(main_area)
        for sarea in sub_areas.copy():
            if len(sarea) > 33:
                sub_areas.remove(sarea)
        flat_areas.extend(sub_areas)
    all_areas.append(flat_areas)

In [7]:
for area in all_areas:
    if area == []:
        print(area)

### One hot encoding

In [8]:
# one-hot encode aol
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(all_areas)

binary_aol_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_aol_df = binary_aol_df.reset_index(drop=True)
processed_df = pd.concat([merged_df.drop('area_of_law', axis=1), binary_aol_df], axis=1)

print(processed_df.head(3))

           casename                                        Coram  themes  \
0   2000_SGCA_1.pdf  [Chao Hick Tin , L P Thean , Yong Pung How]     5.0   
1  2000_SGCA_10.pdf   [Chao Hick Tin , Tan Lee Meng , L P Thean]     5.0   
2  2000_SGCA_11.pdf   [Chao Hick Tin , Tan Lee Meng , L P Thean]     5.0   

  court_level                                             issues  \
0        SGCA  The claim was dismissed with costs by the\nHig...   
1        SGCA  the claim and\nagainst that decision this appe...   
2        SGCA  The appeal \nThe questions which arise in this...   

                                               facts      target  \
0  The facts\nThe appellant is the widow of one T...  Favourable   
1  facts and surrounding circumstances including ...  Favourable   
2  Background \nThe first appellants, a French co...  No outcome   

   "a larger sum being repaid"  "abet"  "an interest in any matter"  ...  \
0                            0       0                            0  ... 

In [9]:
print(processed_df.isna().sum())
# processed_df = processed_df.dropna()

casename                         0
Coram                            0
themes                           0
court_level                      0
issues                           0
                                ..
young offenders]                 0
“any claim  hereunder”           0
“any fire accidentally begin”    0
“charity proceedings”            0
“rash” and “negligent”           0
Length: 1378, dtype: int64


In [10]:
processed_df = processed_df[processed_df['Coram'].apply(lambda x: isinstance(x, list))]

In [11]:
# one-hot encode coram
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['Coram'])

binary_coram_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_coram_df = binary_coram_df.reset_index(drop=True)
processed_df = pd.concat([processed_df.drop('Coram', axis=1), binary_coram_df], axis=1)

print(processed_df.head())

           casename  themes court_level  \
0   2000_SGCA_1.pdf     5.0        SGCA   
1  2000_SGCA_10.pdf     5.0        SGCA   
2  2000_SGCA_11.pdf     5.0        SGCA   
3  2000_SGCA_12.pdf     5.0        SGCA   
4  2000_SGCA_13.pdf    12.0        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the claim and\nagainst that decision this appe...   
2  The appeal \nThe questions which arise in this...   
3  the appeals from the assistant registrar. In h...   
4  the appeal on 24 January 2000 and dismissed it...   

                                               facts        target  \
0  The facts\nThe appellant is the widow of one T...    Favourable   
1  facts and surrounding circumstances including ...    Favourable   
2  Background \nThe first appellants, a French co...    No outcome   
3  Background\nMicrosoft, Adobe and Autodesk are ...  Unfavourable   
4  facts. Mere assertion would not suffice. In ex...  

In [12]:
print(processed_df.isna().sum())

casename                        0
themes                          0
court_level                     0
issues                          0
facts                           0
                               ..
Yeong Zee Kin SAR               0
Yong Pung How                   0
Yong Pung How,                  0
Yong Pung How, Chief Justice    0
Zhuo Wenzhao AR                 0
Length: 1600, dtype: int64


### Topic Modelling

#### Helper functions for Topic Modelling 

In [13]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    legal_stopwords = ('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'mrs', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'slr', 'sgdc', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could', 'next')
    stop_words.update(legal_stopwords)
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words

#### Topic Modelling For Facts

In [14]:
data = []

processed_df['processed_facts'] = processed_df['facts'].apply(preprocess_text)
processed_df.drop(columns=['facts'], inplace=True)
print(processed_df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in processed_df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# print(lda_model.show_topics())

# best_coherence = -1
# best_lda = None
# for num_topics in range(5, 31, 5):
#     # Train LDA model
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                 id2word=dictionary,
#                                                 num_topics=num_topics,
#                                                 random_state=42)
    
#     # Compute coherence score
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model_lda.get_coherence()
    
#     print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
#     if coherence_score > best_coherence:
#         best_coherence = coherence_score
#         best_topic = num_topics
# print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 10 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=10,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)

processed_df['facts_topic'] = topics
processed_df = processed_df.reset_index(drop=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
6788    [fact, accused, low, sze, song, low, year, old...
6789    [fact, giving, opinion, representation, amount...
6790    [fact, party, karan, bagga, litigant, proceedi...
6791                                                   []
6792    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 6793, dtype: object
Finished preprocessing text
Performing topic modelling


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.097276 -0.016151       1        1  16.721484
4      0.114693 -0.064878       2        1  14.106829
2     -0.099169  0.000553       3        1  13.970872
8     -0.075095 -0.046550       4        1  12.350108
7     -0.038790  0.077832       5        1  11.352299
3     -0.052472  0.002464       6        1   9.713008
6      0.146664  0.025190       7        1   6.916085
0      0.066333  0.032049       8        1   6.683349
1      0.016861  0.036738       9        1   5.861817
9      0.018251 -0.047249      10        1   2.324150, topic_info=         Term          Freq         Total Category  logprob  loglift
3430   victim  10072.000000  10072.000000  Default  30.0000  30.0000
814   accused  19613.000000  19613.000000  Default  29.0000  29.0000
825    charge  11014.000000  11014.000000  Default  28.0000  28.0000
608   offence   6626.000000   6626.000000  Default  27.0000  27.0000
3489     drug   6198.000000   6198.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
108   company    230.028830  17783.033307  Topic10  -5.9854  -0.5860
4058      oct    214.741085  10148.227958  Topic10  -6.0542  -0.0938
383      said    203.401799   7334.508190  Topic10  -6.1084   0.1767
133       day    203.021024   8099.824800  Topic10  -6.1103   0.0755
305   payment    200.433471   8555.305517  Topic10  -6.1231   0.0080

[879 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
31086      4  0.984058     aa
21128      2  0.967872  abang
21128      7  0.006913  abang
21128      8  0.013827  abang
21128     10  0.006913  abang
...      ...       ...    ...
11252      4  0.034950   zhou
11252      6  0.004993   zhou
11252      7  0.004993   zhou
11252      9  0.124820   zhou
11252     10  0.679023   zhou

[4411 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 3, 9, 8, 4, 7, 1, 2, 10])

In [15]:
print(processed_df.isna().sum())

casename                        0
themes                          0
court_level                     0
issues                          0
target                          0
                               ..
Yong Pung How,                  0
Yong Pung How, Chief Justice    0
Zhuo Wenzhao AR                 0
processed_facts                 0
facts_topic                     0
Length: 1601, dtype: int64


#### Topic Modelling For Issues

In [16]:
data = []

processed_df['processed_issues'] = processed_df['issues'].apply(preprocess_text)
processed_df.drop(columns=['issues'], inplace=True)
print(processed_df["processed_issues"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for issue in processed_df['processed_issues']:
    for word in issue:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# print(lda_model.show_topics())

# best_coherence = -1
# best_lda = None
# for num_topics in range(5, 31, 5):
#     # Train LDA model
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                 id2word=dictionary,
#                                                 num_topics=num_topics,
#                                                 random_state=42)
#     # Compute coherence score
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model_lda.get_coherence()
    
#     print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
#     if coherence_score > best_coherence:
#         best_coherence = coherence_score
#         best_topic = num_topics
# print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 25 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=25,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)
    
processed_df['issues_topic'] = topics
processed_df = processed_df.reset_index(drop=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [claim, dismissed, cost, high, decision, fook,...
1       [claim, decision, appeal, brought, background,...
2       [appeal, question, arise, appeal, follows, app...
3       [appeal, assistant, registrar, ground, judgmen...
4       [appeal, january, dismissed, give, reason, evi...
                              ...                        
6788    [issue, sub, issue, arise, consideration, whet...
6789    [claim, conspiracy, defraud, fault, ken, sally...
6790    [issue, relating, defence, justification, qual...
6791                                                   []
6792    [background, dispute, towa, commenced, suit, a...
Name: processed_issues, Length: 6793, dtype: object
Finished preprocessing text
Performing topic modelling


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.035941 -0.049708       1        1  7.261030
11    -0.150897  0.004703       2        1  7.050277
24     0.045542  0.064694       3        1  6.596267
18     0.026956 -0.075909       4        1  5.630869
5      0.023587 -0.049514       5        1  5.442718
8      0.033814  0.008932       6        1  5.413387
16    -0.001762  0.040676       7        1  5.353296
17     0.059053  0.055814       8        1  5.032191
14     0.006138 -0.119943       9        1  4.954589
19    -0.098199 -0.045444      10        1  4.783289
4      0.042383 -0.011365      11        1  4.533793
3      0.050018 -0.034648      12        1  4.369737
21    -0.008742 -0.014829      13        1  3.489951
0      0.038097  0.003338      14        1  3.241335
22     0.036502  0.039493      15        1  3.208887
23     0.011285  0.032970      16        1  2.964046
9     -0.077990 -0.003901      17        1  2.956396
6      0.048759  0.012855      18        1  2.910748
1     -0.074140  0.064358      19        1  2.847406
20    -0.041147  0.034758      20        1  2.682661
12    -0.001936  0.009235      21        1  2.632203
10     0.032961  0.043991      22        1  2.287306
2      0.002706 -0.038149      23        1  2.251416
7     -0.034499  0.009330      24        1  1.545799
13    -0.004429  0.018263      25        1  0.560403, topic_info=            Term          Freq          Total Category  logprob  loglift
210      company  74810.000000   74810.000000  Default  30.0000  30.0000
1152     accused  35275.000000   35275.000000  Default  29.0000  29.0000
3699        drug  18466.000000   18466.000000  Default  28.0000  28.0000
2037        mark  21339.000000   21339.000000  Default  27.0000  27.0000
6115  liquidator   9106.000000    9106.000000  Default  26.0000  26.0000
...          ...           ...            ...      ...      ...      ...
286     decision    328.183209   56136.738059  Topic25  -5.8765   0.0423
575        judge    320.811985   51326.979571  Topic25  -5.8992   0.1092
258         cost    312.889869   45560.306642  Topic25  -5.9242   0.2033
937          see    330.991480   79446.733719  Topic25  -5.8680  -0.2965
749        party    326.347909  108392.717653  Topic25  -5.8821  -0.6213

[2157 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
24876      1  0.022267    aa
24876      2  0.003181    aa
24876      4  0.174959    aa
24876      6  0.003181    aa
24876      9  0.015905    aa
...      ...       ...   ...
15320     12  0.061635  zoom
23717      2  0.009903   zul
23717      5  0.009903   zul
23717     10  0.089125   zul
23717     17  0.871443   zul

[20963 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 12, 25, 19, 6, 9, 17, 18, 15, 20, 5, 4, 22, 1, 23, 24, 10, 7, 2, 21, 13, 11, 3, 8, 14])

In [17]:
processed_df.drop(columns=['themes'], inplace=True)
processed_df

,casename,court_level,target,"""a larger sum being repaid""","""abet""","""an interest in any matter""","""any person""","""appeal""","""arising out of""","""available market""",...,Woo Bih Li,Yeong Zee Kin SAR,Yong Pung How,"Yong Pung How,","Yong Pung How, Chief Justice",Zhuo Wenzhao AR,processed_facts,facts_topic,processed_issues,issues_topic
0,2000_SGCA_1.pdf,SGCA,Favourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[fact, widow, tan, geok, tee, deceased, sue, c...",3,"[claim, dismissed, cost, high, decision, fook,...",8
1,2000_SGCA_10.pdf,SGCA,Favourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[fact, surrounding, circumstance, including, a...",5,"[claim, decision, appeal, brought, background,...",18
2,2000_SGCA_11.pdf,SGCA,No outcome,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[background, appellant, french, company, secon...",2,"[appeal, question, arise, appeal, follows, app...",4
3,2000_SGCA_12.pdf,SGCA,Unfavourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[background, microsoft, adobe, autodesk, compa...",7,"[appeal, assistant, registrar, ground, judgmen...",16
4,2000_SGCA_13.pdf,SGCA,Unfavourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[fact, mere, assertion, suffice, exh, said, st...",6,"[appeal, january, dismissed, give, reason, evi...",19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6788,2023_SGHC_95.pdf,SGHC,No outcome,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[fact, accused, low, sze, song, low, year, old...",4,"[issue, sub, issue, arise, consideration, whet...",9
6789,2023_SGHC_96.pdf,SGHC,Unfavourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[fact, giving, opinion, representation, amount...",1,"[claim, conspiracy, defraud, fault, ken, sally...",15
6790,2023_SGHC_97.pdf,SGHC,Unfavourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[fact, party, karan, bagga, litigant, proceedi...",7,"[issue, relating, defence, justification, qual...",20
6791,2023_SGHC_98.pdf,SGHC,Favourable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0


### splitting

In [18]:
print(processed_df)

              casename court_level        target  "a larger sum being repaid"  \
0      2000_SGCA_1.pdf        SGCA    Favourable                            0   
1     2000_SGCA_10.pdf        SGCA    Favourable                            0   
2     2000_SGCA_11.pdf        SGCA    No outcome                            0   
3     2000_SGCA_12.pdf        SGCA  Unfavourable                            0   
4     2000_SGCA_13.pdf        SGCA  Unfavourable                            0   
...                ...         ...           ...                          ...   
6788  2023_SGHC_95.pdf        SGHC    No outcome                            0   
6789  2023_SGHC_96.pdf        SGHC  Unfavourable                            0   
6790  2023_SGHC_97.pdf        SGHC  Unfavourable                            0   
6791  2023_SGHC_98.pdf        SGHC    Favourable                            0   
6792  2023_SGHC_99.pdf        SGHC    Favourable                            0   

      "abet"  "an interest 

In [20]:
print(processed_df.isna().sum())

casename                       0
court_level                    0
target                         0
"a larger sum being repaid"    0
"abet"                         0
                              ..
Zhuo Wenzhao AR                0
processed_facts                0
facts_topic                    0
processed_issues               0
issues_topic                   0
Length: 1601, dtype: int64


In [22]:
X = processed_df.drop(columns=['target'])
y = processed_df['target']

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, remaining_index in stratified_split.split(X, y):
    X_train, X_test_val = X.iloc[train_index], X.iloc[remaining_index]
    y_train, y_test_val = y.iloc[train_index], y.iloc[remaining_index]

#balanced dataset (target variable was imbalanced Favourable 5006 Unfavourable 2523 No outcome 984)
#randomly found one online, can be changed -> need to check am i doing this right 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

#split further from X_test_val into X_val and X_test
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

### Feature Engineering

In [ ]:
## One hot encoding
## vector embedding
## pipeline

### Modeling

In [ ]:
# Perform modelling

### Evaluation

In [ ]:
## Perform Evaluation